# Lyman-$\alpha$ Equivalent Width Curve of Growth

Figure 2.7 from Chapter 2 of *Interstellar and Intergalactic Medium* by Ryden & Pogge, 2021, 
Cambridge University Press.

Plot of the curve of growth for Hi Ly$\alpha$ with $b=10\,km\,s^{-1}$, corresponding
to $T\approx6000\,K$. The equivalent width is divided by $b_{\lambda}=b\lambda_0/c\approx0.04\,$Angstroms. 

We use a numerical implementation of the Voigt function as the real part of the 
[Faddeeva function](https://en.wikipedia.org/wiki/Faddeeva_function) $w(z)$ in the SciPy specials package, 
function wofz().  We integrate the numerical absorption-line profile with a Simpson's rule integrator from
the SciPy integrate package.  This is good enough for our purposes here.

The linear, flat, and square-root parts of the curve of growth are labeled.

In [ ]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, LogLocator, NullFormatter, FormatStrFormatter

from scipy import special
from scipy import integrate

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)

## Standard Plot Format

Setup the standard plotting format and make the plot. Fonts and resolution adopted follow CUP style.

In [ ]:
figName = 'Fig2_7'

# graphic aspect ratio = width/height

aspect = 4.0/3.0 # 4:3

# Text width in inches - don't change, this is defined by the print layout

textWidth = 6.0 # inches

# output format and resolution

figFmt = 'png'
dpi = 600

# Graphic dimensions 

plotWidth = dpi*textWidth
plotHeight = plotWidth/aspect
axisFontSize = 10
labelFontSize = 6
lwidth = 0.5
axisPad = 5
wInches = textWidth 
hInches = wInches/aspect

# Plot filename

plotFile = f'{figName}.{figFmt}'

# LaTeX is used throughout for markup of symbols, Times-Roman serif font

plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Times-Roman'],'weight':'bold','size':'16'})

# Font and line weight defaults for axes

matplotlib.rc('axes',linewidth=lwidth)
matplotlib.rcParams.update({'font.size':axisFontSize})

# axis and label padding

plt.rcParams['xtick.major.pad'] = f'{axisPad}'
plt.rcParams['ytick.major.pad'] = f'{axisPad}'
plt.rcParams['axes.labelpad'] = f'{axisPad}'

## Lyman-$\alpha$ absorption calculation

Formal implementation using Faddeeva or Kramp function w(z)
implemented in scipy.special as wofz(). plucked off SciPy-User
from archives via Google.  For a detailed derivation, the
Wikipedia articles are good:
 * [Voigt Profile](http://en.wikipedia.org/wiki/Voigt_profile)
 * [Faddeeva Function](http://en.wikipedia.org/wiki/Faddeeva_function)

The Voight function is the real part of the Faddeeva function.  We're slightly finessing the normalization
here for illustration purposes.

The Lyman-$\alpha$ line will be computed for the following properties:
 * Optical depth at line center of $\log_{10}(\tau_0)$= -2.0 to +7 in 0.1 steps
 * Doppler parameter $b=10\,km\,s^{-1}$.
 * Lorentzian natural width for HI Lyman $\alpha$ of $\gamma=2.458\times10^{-05}\,$Angstroms
 * Line center $\lambda_0=1215.67\,$Angstroms
 * Integration limits are $\pm30\,$Angstroms in 0.01 Angstrom steps

In [ ]:
def H(a,u):
    return special.wofz(u + 1j*a).real

# Constants

c = 2.99792458e05  # speed of light in km/sec (CODATA 2018)

# pixel size Angstroms and integration width in pixels

dLam = 0.01 # Angstroms
wPix = 3000 # pixels

# Properties of HI Lyman-alpha, mix of constants and parameters

lam0 = 1215.67    # Lyman-alpha in Angstroms
b = 10.0          # Doppler width in km/sec
blam = lam0*b/c   # Doppler width in Angstroms
gamma = 2.458e-05 # Natural width for HI Lyman Alpha in Angstroms
a = gamma/blam    # line a factor

# Range of optical depths

minLogTau = -2.0
maxLogTau = 7.0
dLogTau = 0.1
nLogTau = int((maxLogTau-minLogTau)/dLogTau)
logTau0 = np.linspace(minLogTau,maxLogTau,nLogTau)

# wavelength range and u parameter for the Voight-Hjerting function

minLam = lam0 - dLam*wPix
maxLam = lam0 + dLam*wPix
numPix = 2*wPix+1
lam = np.linspace(minLam,maxLam,num=numPix)
u = (lam-lam0)/blam

# For each optical depth, compute the line profile and integrate it

tau0 = []
eqW = []
eqWblam = []
for lt0 in logTau0:
    t0 = 10.0**lt0
    tau0.append(t0)
    sumEW = 0.0
    integrand = 1.0 - np.exp(-t0*H(a,u))
    intEW = integrate.simps(integrand,lam)
    eqW.append(intEW)
    eqWblam.append(intEW/blam)

# x-axis limits

minTau = 10.0**(minLogTau)
maxTau = 10.0**(maxLogTau)

# y-axis limits

eqMin = np.min(eqWblam)
eqMax = np.max(eqWblam)
minWb = eqMin
maxWb = eqMax + 0.1*(eqMax-eqMin)


### Make the Plot

Plot the curve of growth $W_{\lambda}/b_{\lambda}$ as a function of $\tau_0$ on a log-log scale.

In [ ]:
fig,ax = plt.subplots()

fig.set_dpi(dpi)
fig.set_size_inches(wInches,hInches,forward=True)

ax.tick_params('both',length=6,width=lwidth,which='major',direction='in',top='on',right='on')
ax.tick_params('both',length=3,width=lwidth,which='minor',direction='in',top='on',right='on')

plt.xlim(minTau,maxTau)
ax.set_xscale('log')
ax.set_xticks([.01,.1,1,10,100,1000,1.0e4,1.0e5,1.0e6,1.0e7])
ax.set_xticklabels(['0.01','0.1','1','10','100','1000','10$^4$','10$^5$','10$^6$','10$^7$'])
plt.xlabel(r'$\tau_0$')

plt.ylim(minWb,maxWb)
ax.set_yscale('log')
ax.set_yticks([0.1,1.0,10,100])
ax.set_yticklabels(['0.1','1','10','100'])
plt.ylabel(r'$W_{\lambda}/b_{\lambda}$')

# plot the curves

plt.plot(tau0,eqWblam,'-',color='black',lw=1.5,zorder=10)

# Label the main parts of the CoG

plt.text(0.1,0.3,'Linear',rotation=58.0,fontsize=axisFontSize,ha='center',va='center')
plt.text(200.0,6.5,'Flat',rotation=10.0,fontsize=axisFontSize,ha='center',va='center')
plt.text(2.2e5,44,'Square-root',rotation=39.0,fontsize=axisFontSize,ha='center',va='center')

# plot and file

plt.plot()
plt.savefig(plotFile,bbox_inches='tight',facecolor='white')